In [12]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split as sk_split
from sklearn import linear_model

np.set_printoptions(threshold='nan')

In [10]:
np.zeros(5)

array([ 0.,  0.,  0.,  0.,  0.])

In [16]:
training_data = pd.read_csv('datasets/tweets.csv', delimiter=',')
testing_data = pd.read_csv('datasets/tweets2.csv', delimiter=',')

clinton_data_1 = training_data[training_data.handle == "HillaryClinton"]["text"].as_matrix()
trump_data_1 = training_data[training_data.handle == "realDonaldTrump"]["text"].as_matrix()

clinton_data_2 = testing_data[testing_data.handle == "HillaryClinton"]["text"].as_matrix()
trump_data_2 = testing_data[testing_data.handle == "realDonaldTrump"]["text"].as_matrix()

clinton_data = np.concatenate((clinton_data_1, clinton_data_2), axis=0)
trump_data = np.concatenate((trump_data_1, trump_data_2), axis=0)

clinton_dataset = []
trump_dataset = []

def loadCharsFromTxt(text, dataset):
    image = np.zeros((140, 36))
    words = text.lower().split()
    
    charCount = 0
    spaceIndex = 0
    for word in words:
        if "t.co" in word:
            continue
        else:
            image[charCount, spaceIndex] = 1
            charCount += 1
            for char in word:
                index = None
                if char.isalpha():
                    index = ord(char) - ord('a')
                elif char.isdigit():
                    index = ord(char) - ord('0') + 26
                if index:
                    image[charCount, index] = 1
                    charCount += 1

    dataset.append(image)

for text in clinton_data:
    loadCharsFromTxt(text, clinton_dataset)

for text in trump_data:
    loadCharsFromTxt(text, trump_dataset)
    
clinton_y = np.zeros(len(clinton_dataset))
trump_y = np.full(len(trump_dataset), 1)

x_data = np.concatenate((clinton_dataset, trump_dataset), axis=0)
y_data = np.concatenate((clinton_y, trump_y), axis=0)

x_train, x_test, y_train, y_test = sk_split(x_data, y_data, test_size = 0.25, random_state = 42)

print x_train.shape
print y_train.shape

model = linear_model.LogisticRegression()
model.fit(x_train, y_train)
print model.score(x_test, y_test)

(9666, 140, 36)
(9666,)


ValueError: Found array with dim 3. Estimator expected <= 2.